In [ ]:
#shell of notebook

In [2]:
# rachel will start
def display_drop_down(options_list):
    """
    displays a drop down for the items in options list, collects choice from the user and returns a string with
    their choice
    
    Requirements: Sector_group_name, Sector_name, Industry_name
    """
    return "some_company"

In [3]:
#rachel with start
def display_company_report(clean_company_name):
    """
    given a clean company name, show a display report of the company 
    
    SIC 
    LOC
    F/D % percent of companies that are abroad vs. domestic
    
    
    Requirements: Corpwatch Company data, company relationship data

    """
    display_breakdown(clean_company_name)
    display_hierarchy(clean_company_name)
    
    company = input('')
    display_company_report(company)
    #display something
    #return the report

In [7]:
#prakar
def search_by_sector(sector_name):
    """
    returns list of a parent companies in that sub sector, ranked by subsidiaries
    """
    return []

In [ ]:
def display_breakdown(clean_company_name):
    '''
    uses logic to determine if it should display (e.g. is it a parent)
    
    if so, shows Pie chart of sector breakdown
    
    shows list of sectors they are in
    '''
    return []

In [ ]:
def display_hierarchy(clean_company_name):
    '''
    uses logic to determine if it should display (e.g. does it have at least a parent or a child)
    
    if so, show table of parent (if applicable)
    show table of children (if applicable)
    '''
    return []

In [ ]:
from edgar import Company, TXTML, Edgar

import string
edgar= Edgar()

class CompanyNode:
    def __init__(self, name, location, parent = None, level = 0):
        print(name)
        self.name = self.convert_name_to_SEC_standards(name)
#       self.corp_type()
        
        self.location = location

        self.parent = parent
        self.level = level
        self.child = {}
        self.cik = None
        self.status = None

        
        
        
        
        
        self.SEC = self.eval_SEC() #indicates whether the company can be found on SEC
        if self.SEC == True:
            self.Search_Edgar()
        
        print(self.name, self.status)
        self.multiple_companies = None
        
        
        
    
    #used to convert name into format that is more likely to be picked up by SEC/Package Search
    def convert_name_to_SEC_standards(self, name):
        if len(name.split()) < 2:
            return name.upper()
        else:
            name_split = [x.upper() for x in name.split()]
            #print(name_split)
            #print(name_split[-2:])
            #Company Transform
            for i in range(2):
                #removes the word "the" if its at the start of the document
                if "THE" == name_split[0]:
                    name_split.pop(0)
                #removes punctuation if its not an LLC
                if "LLC" not in name_split[-1]:
                    name_split[i-2] = name_split[i-2].translate(str.maketrans('', '', string.punctuation))

                #converts words
                #converts company to co
                if "COMPANY" in name_split[i-2]:
                    name_split[i-2] = "CO"
                #converts corporation to corp
                elif "CORPORATION" in name_split[i-2]:
                    name_split[i-2] = "CORP"
                elif "INCORPORATED" in name_split[i-2]:
                    name_split[i-2] = "INC"
            #grab the corporate type from the end of the list
    #         self.corp_type = name_split[-1]
            SEC_name = ' '.join(name_split)
            #print(SEC_name)
            #returns reformatted name and corporation type
            return SEC_name
             
    
    def eval_corp_type(self):
        corp_check = self.name.split()
        print(corp_check[-1])
        return corp_check[-1]
    
#     def set_SIC(self, SIC):
#         self.sic = SIC
    
#

    def find_parent(self):
        if self.parent == None:
            return(self.name)
        else:
            return self.parent.find_parent()


    def eval_SEC(self):
        states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
                  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
                  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
                  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
                  "Nebraska","Nevada","New Hampshire","New Jersey","New Mexico","New York",
                  "North Carolina","North Dakota","Ohio","Oklahoma","Oregon","Pennsylvania",
                  "Rhode Island","South Carolina","South Dakota","Tennessee","Texas","Utah",
                  "Vermont","Virginia","Washington","West Virginia","Wisconsin","Wyoming"]        
        
        
        
        #deal with this so we can allow for permutations (e.g. USA - Delaware
        if self.location in states:
            return True
        else:
            self.status = (4, 'Non-US Company')
            return False
    
    def Search_Edgar(self):    
        #right now we have two things we can do in either order. We can either search for the name in edgar first(1 second), then see how big the list is
        #then search for the CIK(1 millisecond) if the length is 1. Or we can search for the CIK first (1 millisecond), then search for the name to find out
        #why it failed. The difference that in one we're checking first, then finding out why it failed (fast CIK then slower search if it fails), 
        #while in the other we're running the slower one first to find if it will work, then running the faster one if it meets the right conditions
        #(slower search then fast CIK if it fails).

        #after doing some math, they will be equally efficient if 4/5th of the searches are errors, fast then slower will be better if <4/5th are errors,
        #while slow then fast will be betterif >4/5th are errors. We expect a high number of failures, but perhaps not quite so many. Will do some more testing, then decide


        #checks for cik and gets it if we can
        try:
            self.cik = edgar.get_cik_by_company_name(self.name)
        except: #if it fails find out why   
            possible_companies = edgar.find_company_name(self.name)
            if len(possible_companies) == 0:
                self.status = (1,'Company is not in SEC/Name Misspelled')
                return
            elif len(possible_companies) > 1:
                self.status = (2, 'Multiple search results found')
                #sets this so we can better check failure conditions
                self.multiple_companies = possible_companies
                return
            elif len(possible_companies) == 1:
                self.status = (3, 'Return one result, better check what went wrong')
                return
        #other way in case we need to change later
    #     possible_companies = edgar.find_company_name(self.name)
    #     if len(possible_companies) == 0
    #         cik = edgar.get_cik_by_company_name(possible_companies[0])

        #only hits next step if cik works
        if self.cik != None:
            #create company
            company = Company(self.name, self.cik)

            #decide between grabbing 10-K (2.4 seconds) first to see if its there, or grabbing EX21( 1.5 seconds) to see if its there


            EX21 = company.get_document_type_from_10K("EX-21")
            if len(EX21) == 0:
                EX21 = company.get_document_type_from_10K("EX-21.1")
            if len(EX21) == 0:
                self.status = (5,"Cannot find EX-21.1 for Company")
                #build in follow up check to see if 10-K is there
            else:
                #pass in EX21 to parsing document
                try:
                    body_node = EX21[0][0]
                    print(type(body_node))

        #             tree = body_node.getchildren()



                    #there is def. a better way to do this
                    subs = []
                    loc = []
                    print(type(body_node.iter("tr")))

                    for i in body_node.iter("tr"):
                        count = 0
                        for j in i.iter("p"):
                            if count == 0 and j.text_content != "Company Name":
                                subs.append(j.text_content())
                            elif count == 1 and j.text_content != "Domicile\xa0or\xa0State\xa0of\xa0Incorporation":
                                loc.append(j.text_content())
                            count+=1
#                     subs.reverse()
#                     loc.reverse()
                    print(len(loc))

                    print(len(subs))
                    for i in range(len(subs)):
                        print(i)
                        if subs[i] not in self.child:
                            self.child[subs[i]] = CompanyNode(subs[i], loc[i], self, self.level+1)
                    print(subs)
                    #print(loc)
                    self.status = (0, "Success")
                except:
                    self.status = (6,"Cannot Parse EX 21")
                
#test = CompanyNode("BERKSHIRE HATHAWAY INC", "Delaware")
#print(test.status)

In [10]:
# Max is going to write this to start with


def company_suggestions(company_name):
    '''
    if name isn't clean, then search name+wildcard to return list of companies, ranked by number of children.
    can remove
    '''
    company = input('')
    display_company_report(company) #navigates to company report if selected

    return [] #ranked list of company suggestions

In [9]:
result = 

search by company or sector


In [6]:
if result == 'sector':
    #show sector group
    
    sector_compnay_list = search_by_sector(sector_name): #returns all companies in sector
    company = display_dropdown(sector_compnay_list) #displays companies with subsidiaries 
    display_company_report(company) #navigates to company report if selected
    #show sector name
    #show industry name
    

SyntaxError: invalid syntax (<ipython-input-6-54b4f85b485d>, line 3)